In [16]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
spark = SparkSession.builder.master("local[*]").appName('').getOrCreate()

In [17]:
schema = StructType([
    StructField("ano", LongType(), True),
    StructField("codigo", StringType(), True),
    StructField("des_localidade_id", StringType(), True),
    StructField("des_localidade_nome", StringType(), True),
    StructField("des_localidade_uf", StringType(), True),
    StructField("des_municipio_id", StringType(), True),
    StructField("des_municipio_nome", StringType(), True),
    StructField("empresa", StringType(), True),
    StructField("empresa_cnpj", StringType(), True),
    StructField("empresa_situacao", StringType(), True),
    StructField("empresa_tipo", StringType(), True),
    StructField("ida_idoso_desconto", StringType(), True),
    StructField("ida_idoso_gratis", StringType(), True),
    StructField("ida_jovem_desconto", StringType(), True),
    StructField("ida_jovem_gratis", StringType(), True),
    StructField("ida_pagantes", StringType(), True),
    StructField("ida_passelivre", StringType(), True),
    StructField("km", StringType(), True),
    StructField("km_total", StringType(), True),
    StructField("linha", StringType(), True),
    StructField("linha_id", StringType(), True),
    StructField("lugares_idas", StringType(), True),
    StructField("lugares_voltas", StringType(), True),
    StructField("mes", StringType(), True),
    StructField("ori_des_localidade_nome", StringType(), True),
    StructField("ori_localidade_id", StringType(), True),
    StructField("ori_localidade_nome", StringType(), True),
    StructField("ori_localidade_uf", StringType(), True),
    StructField("ori_municipio_id", StringType(), True),
    StructField("ori_municipio_nome", StringType(), True),
    StructField("pax_gratis_descontos", StringType(), True),
    StructField("pax_idoso_desconto", StringType(), True),
    StructField("pax_idoso_gratis", StringType(), True),
    StructField("pax_jovem_desconto", StringType(), True),
    StructField("pax_jovem_gratis", StringType(), True),
    StructField("pax_pagantes", StringType(), True),
    StructField("pax_passelivre", StringType(), True),
    StructField("pax_total", StringType(), True),
    StructField("prefixo", StringType(), True),
    StructField("secao_id", StringType(), True),
    StructField("sequencial", StringType(), True),
    StructField("servico_ambito", StringType(), True),
    StructField("servico_tipo", StringType(), True),
    StructField("sisdap_fim", StringType(), True),
    StructField("sisdap_inicio", StringType(), True),
    StructField("viagem_idas", StringType(), True),
    StructField("viagem_voltas", StringType(), True),
    StructField("volta_idoso_desconto", StringType(), True),
    StructField("volta_idoso_gratis", StringType(), True),
    StructField("volta_jovem_desconto", StringType(), True),
    StructField("volta_jovem_gratis", StringType(), True),
    StructField("volta_pagantes", StringType(), True),
    StructField("volta_passelivre", StringType(), True)])

In [18]:
# tb = spark.read.option("inferSchema", "true").parquet("intermunicipal_sc/ref/parquet")
tb = spark.read.schema(schema).parquet("intermunicipal_sc/ref/parquet")
# tb = spark.read.parquet("intermunicipal_sc/ref/parquet")
tb.createOrReplaceTempView("tb")

dicionario = spark.read.option("delimiter", ";").option("header", "true").option("inferSchema", "true").option("encoding", "utf-8").csv("./dicionario_deter_sie.csv")
dicionario.createOrReplaceTempView("dicionario")

In [19]:
df_join = spark.sql("""

SELECT
tb.ano AS `ano`,
tb.empresa AS `empresa`,
prefixo,
CONCAT(ori_municipio_nome, '-', des_municipio_nome) as linha_ori_des_sie,
CONCAT(loc_ori.SIG_CONCAT_UPPER, '-', loc_des.SIG_CONCAT_UPPER) AS localidade_ori_des_sig,
km,
pax_total AS `passageiros_total`
FROM tb
LEFT JOIN dicionario AS loc_ori ON tb.ori_localidade_nome = loc_ori.SIE_NOME
LEFT JOIN dicionario AS loc_des ON tb.des_localidade_nome = loc_des.SIE_NOME
LEFT JOIN dicionario AS mun_ori ON tb.ori_municipio_nome = mun_ori.SIE_NOME
LEFT JOIN dicionario AS mun_des ON tb.des_municipio_nome = mun_des.SIE_NOME

""").show(10, False)

# WHERE ano = '2000' AND (pax_total > 0) AND (loc_ori.SIG_CONCAT_UPPER LIKE 'GUATAMBÚSC' OR loc_des.SIG_CONCAT_UPPER LIKE 'GUATAMBÚSC')

# WHERE ano = '2000' AND (pax_total > 0) AND (loc_ori.SIG_CONCAT_UPPER LIKE 'CHAPECÓSC' OR loc_des.SIG_CONCAT_UPPER LIKE 'CHAPECÓSC')

# df_join.write.option("delimiter", ",").option("header", "true").option("encoding", "utf-8").csv("./testecsv/")

# df_join.write.partitionBy("empresa").parquet("./test_joined")

+----+-----------------------------+-------+-----------------------------+-----------------------------------------+------+-----------------+
|ano |empresa                      |prefixo|linha_ori_des_sie            |localidade_ori_des_sig                   |km    |passageiros_total|
+----+-----------------------------+-------+-----------------------------+-----------------------------------------+------+-----------------+
|2008|1-EMP STO ANJO DA GUARDA LTDA|16-0   |BALNEARIO CAMBORIU-JOINVILLE |BALNEÁRIO CAMBORIÚSC-ITAJAÍSC            |11,00 |10140            |
|2008|1-EMP STO ANJO DA GUARDA LTDA|16-0   |BALNEARIO CAMBORIU-JOINVILLE |BALNEÁRIO CAMBORIÚSC-NAVEGANTESSC        |26,00 |173              |
|2008|1-EMP STO ANJO DA GUARDA LTDA|16-0   |BALNEARIO CAMBORIU-JOINVILLE |BALNEÁRIO CAMBORIÚSC-BALNEÁRIO PIÇARRASSC|36,00 |3673             |
|2008|1-EMP STO ANJO DA GUARDA LTDA|16-0   |BALNEARIO CAMBORIU-JOINVILLE |BALNEÁRIO CAMBORIÚSC-BARRA VELHASC       |43,00 |620              |
|2008|

In [20]:
spark.sql("""

SELECT ano AS Ano, empresa AS Empresa, CONCAT(ori_municipio_nome, '-', des_municipio_nome) AS Linha, ori_des_localidade_nome AS `Origem-destino`, pax_total AS `Total de passageiros`
FROM tb
WHERE ano = 2019
AND (empresa LIKE '23%' OR empresa LIKE '1505%')
AND pax_total > 10000
AND servico_ambito = 'RODOVIARIO'
ORDER BY pax_total DESC

""").show(10, False)

+----+---------------------------------+-----------------------+---------------------+--------------------+
|Ano |Empresa                          |Linha                  |Origem-destino       |Total de passageiros|
+----+---------------------------------+-----------------------+---------------------+--------------------+
|2019|23-REUNIDAS S/A-TRANSP COLETIVOS |CONCORDIA -CHAPECO     |CONCORDIA-CHAPECO    |18585               |
|2019|23-REUNIDAS S/A-TRANSP COLETIVOS |LAGES -FLORIANOPOLIS   |LAGES-FLORIANOPOLIS  |14683               |
|2019|23-REUNIDAS S/A-TRANSP COLETIVOS |FLORIANOPOLIS -BLUMENAU|BRUSQUE-BLUMENAU     |13541               |
|2019|23-REUNIDAS S/A-TRANSP COLETIVOS |CONCORDIA -CHAPECO     |SEARA-CHAPECO        |12997               |
|2019|23-REUNIDAS S/A-TRANSP COLETIVOS |LAGES -CURITIBANOS     |LAGES-CORREIA PINTO  |11451               |
|2019|23-REUNIDAS S/A-TRANSP COLETIVOS |MAFRA -JOINVILLE       |MAFRA-S BENTO DO SUL |11017               |
+----+----------------------

In [21]:
tb.select(
    "ano",
    "empresa",
    "prefixo",
    concat_ws("-", "ori_municipio_nome", "des_municipio_nome").alias("linha_ori_des_sie"),
    concat_ws("-", "ori_localidade_nome", "des_localidade_nome").alias("secao_ori_des_sie"),
    "pax_total"
).show(10, False)

+----+-----------------------------+-------+-----------------------------+---------------------------+---------+
|ano |empresa                      |prefixo|linha_ori_des_sie            |secao_ori_des_sie          |pax_total|
+----+-----------------------------+-------+-----------------------------+---------------------------+---------+
|2008|1-EMP STO ANJO DA GUARDA LTDA|16-0   |BALNEARIO CAMBORIU-JOINVILLE |BALN CAMBORIU-ITAJAI       |10140    |
|2008|1-EMP STO ANJO DA GUARDA LTDA|16-0   |BALNEARIO CAMBORIU-JOINVILLE |BALN CAMBORIU-S NICOLAU    |173      |
|2008|1-EMP STO ANJO DA GUARDA LTDA|16-0   |BALNEARIO CAMBORIU-JOINVILLE |BALN CAMBORIU-PICARRAS     |3673     |
|2008|1-EMP STO ANJO DA GUARDA LTDA|16-0   |BALNEARIO CAMBORIU-JOINVILLE |BALN CAMBORIU-ITAJUBA 2    |620      |
|2008|1-EMP STO ANJO DA GUARDA LTDA|16-0   |BALNEARIO CAMBORIU-JOINVILLE |BALN CAMBORIU-BARRA VELHA  |887      |
|2008|1-EMP STO ANJO DA GUARDA LTDA|16-0   |BALNEARIO CAMBORIU-JOINVILLE |BALN CAMBORIU-ITAPOCU 

In [22]:
# tb.printSchema()
tb.select(
    "ano",
    "empresa",
    struct("prefixo", concat_ws("-", "ori_municipio_nome", "des_municipio_nome").alias("linha_ori_des_sie")).alias("linha"),
    concat_ws("-", "ori_localidade_nome", "des_localidade_nome").alias("secao_ori_des_sie"),
    "pax_total"
).show(10, False)

+----+-----------------------------+-------------------------------------+---------------------------+---------+
|ano |empresa                      |linha                                |secao_ori_des_sie          |pax_total|
+----+-----------------------------+-------------------------------------+---------------------------+---------+
|2008|1-EMP STO ANJO DA GUARDA LTDA|{16-0, BALNEARIO CAMBORIU-JOINVILLE }|BALN CAMBORIU-ITAJAI       |10140    |
|2008|1-EMP STO ANJO DA GUARDA LTDA|{16-0, BALNEARIO CAMBORIU-JOINVILLE }|BALN CAMBORIU-S NICOLAU    |173      |
|2008|1-EMP STO ANJO DA GUARDA LTDA|{16-0, BALNEARIO CAMBORIU-JOINVILLE }|BALN CAMBORIU-PICARRAS     |3673     |
|2008|1-EMP STO ANJO DA GUARDA LTDA|{16-0, BALNEARIO CAMBORIU-JOINVILLE }|BALN CAMBORIU-ITAJUBA 2    |620      |
|2008|1-EMP STO ANJO DA GUARDA LTDA|{16-0, BALNEARIO CAMBORIU-JOINVILLE }|BALN CAMBORIU-BARRA VELHA  |887      |
|2008|1-EMP STO ANJO DA GUARDA LTDA|{16-0, BALNEARIO CAMBORIU-JOINVILLE }|BALN CAMBORIU-ITAPOCU 

In [23]:
# tb.groupBy("empresa").sum("pax_total").show(10, False)